In [ ]:
pip install opencv-python pillow numpy scikit-learn tensorflow torch torchvision

In [ ]:
import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras import layers, models
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image
from skimage.metrics import peak_signal_noise_ratio as psnr
import matplotlib.pyplot as plt 

In [ ]:
def load_images(directory, size=(128, 128)):
    img_list = []
    for file in os.listdir(directory):
        path = os.path.join(directory, file)
        image = cv2.imread(path)
        if image is not None:
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            image = cv2.resize(image, size)
            image = image / 255.0
            img_list.append(image)
    return img_list

In [ ]:
low_quality_path = 'C:/Users/Kunal /Downloads/Train/low'
high_quality_path = 'C:/Users/Kunal/Downloads/Train/high'
image_size = (128, 128)

In [ ]:
# Load the Images
low_quality_images = load_images(low_quality_path, image_size)
high_quality_images = load_images(high_quality_path, image_size)

In [ ]:
train_low, test_low, train_high, test_high = train_test_split(
    low_quality_images, high_quality_images, test_size=0.2, random_state=42
)

In [ ]:
# Preprocessing Function for TensorFlow
def image_preprocessing(image):
    return tf.image.convert_image_dtype(image, tf.float32)

# Create TensorFlow Datasets
train_low_ds = tf.data.Dataset.from_tensor_slices(train_low).map(image_preprocessing).batch(32)
train_high_ds = tf.data.Dataset.from_tensor_slices(train_high).map(image_preprocessing).batch(32)
test_low_ds = tf.data.Dataset.from_tensor_slices(test_low).map(image_preprocessing).batch(32)
test_high_ds = tf.data.Dataset.from_tensor_slices(test_high).map(image_preprocessing).batch(32)

In [ ]:
# Combine Noisy and Clean Images for Training
training_dataset = tf.data.Dataset.zip((train_low_ds, train_high_ds))
testing_dataset = tf.data.Dataset.zip((test_low_ds, test_high_ds))

In [ ]:
# Model Creation Function
def build_denoising_network(input_shape):
    net = models.Sequential()
    net.add(layers.Input(shape=input_shape))
    net.add(layers.Conv2D(64, (3, 3), activation='relu', padding='same'))
    net.add(layers.MaxPooling2D((2, 2), padding='same'))
    net.add(layers.Conv2D(32, (3, 3), activation='relu', padding='same'))
    net.add(layers.MaxPooling2D((2, 2), padding='same'))
    net.add(layers.Conv2D(32, (3, 3), activation='relu', padding='same'))
    net.add(layers.UpSampling2D((2, 2)))
    net.add(layers.Conv2D(64, (3, 3), activation='relu', padding='same'))
    net.add(layers.UpSampling2D((2, 2)))
    net.add(layers.Conv2D(3, (3, 3), activation='sigmoid', padding='same'))
    return net
# Define the Model
input_dimensions = (128, 128, 3)
denoising_model = build_denoising_network(input_dimensions)
denoising_model.compile(optimizer='adam', loss='mean_squared_error')
denoising_model.summary()

In [ ]:
# Train the Model
training_history = denoising_model.fit(training_dataset, epochs=50, validation_data=testing_dataset)

In [ ]:
# Predict with the Model
output_images = denoising_model.predict(test_low_ds)

In [ ]:
# Calculate PSNR
psnr_scores = [psnr(test_high[i], output_images[i]) for i in range(len(test_high))]
mean_psnr = np.mean(psnr_scores)
print(f'Average PSNR: {mean_psnr}')